In [31]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import joblib
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN, InputLayer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from scipy.interpolate import interp1d
import math
from features import implied_probability, american_to_decimal, fixed_length, split_for_datetime, time_for_75, timeInput

In [32]:
df = pd.read_csv("timestampsDatetimeA.csv", header=None, names=range(231))
df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True, inplace=True)
df.head()

C:\Users\maxbr\AppData\Local\Temp\ipykernel_3928\2356563460.py:1: DtypeWarning: Columns (206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("timestampsDatetimeA.csv", header=None, names=range(231))


,fighter,event,date,opponent,timestep-1,timestep-2,timestep-3,timestep-4,timestep-5,timestep-6,...,timestep-218,timestep-219,timestep-220,timestep-221,timestep-222,timestep-223,timestep-224,timestep-225,timestep-226,timestep-227
0,Mansur Abdul-Malik,UFC,2025-02-23,Nick Klein,2025-02-13 14:17:00 - price: -549,2025-02-13 20:33:00 - price: -549,2025-02-14 18:09:00 - price: -562,2025-02-14 18:13:00 - price: -532,2025-02-14 19:09:00 - price: -569,2025-02-14 19:11:00 - price: -569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Daichi Abe,UFC Fight Night 132: Cowboy vs. Edwards,2018-06-23,Li Jingliang,2018-06-13 12:09:00 - price: +240,2018-06-13 12:21:00 - price: +325,2018-06-13 13:42:00 - price: +315,2018-06-13 15:24:00 - price: +323,2018-06-13 16:18:00 - price: +320,2018-06-13 17:06:00 - price: +313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Adajar,Road to UFC 4,2022-06-10,Han Seul Kim,2022-06-07 10:00:00 - price: -120,2022-06-07 10:18:00 - price: +105,2022-06-07 22:02:00 - price: -105,2022-06-08 13:08:00 - price: +100,2022-06-08 13:26:00 - price: +115,2022-06-08 13:50:00 - price: +118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Shamil Abdurakhimov,UFC 283,2023-01-21,Jailton Almeida,2023-01-06 10:00:00 - price: +600,2023-01-06 13:20:00 - price: +583,2023-01-07 19:48:00 - price: +588,2023-01-08 15:40:00 - price: +585,2023-01-08 23:16:00 - price: +593,2023-01-08 23:54:00 - price: +596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hamdy Abdelwahab,UFC,2025-02-01,Jamal Pogues,2025-01-14 07:53:00 - price: +105,2025-01-14 12:39:00 - price: +113,2025-01-15 08:41:00 - price: +110,2025-01-16 12:13:00 - price: +111,2025-01-16 14:27:00 - price: +111,2025-01-16 15:57:00 - price: +112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
X = np.empty((0, 20, 3))
y = np.empty((0, 1))

for index, row in df.iterrows():
    dfTest = df.loc[[index]]

    dfTest = dfTest.dropna(axis=1)

    timesequence = timeInput(dfTest)

    dfTest = dfTest.iloc[:, 4:]

    split_name = time_for_75(dfTest)
    split_index = dfTest.columns.get_loc(split_name)


    dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
            if isinstance(x, str) and 'price: ' in x else x
        )
    

    if split_index <8:
        continue
    
    target = float(dfTest.iloc[0, -1])
    sequence = dfTest.iloc[0, :split_index].values
    sequence = np.array(sequence, dtype=float)

    time_since_open_seq = timesequence.iloc[0, :split_index].values
    time_till_close_seq = timesequence.iloc[1, :split_index].values

    time_since_open_seq = np.array(time_since_open_seq, dtype=float)
    time_till_close_seq = np.array(time_till_close_seq, dtype=float)

    last_value = sequence[-1]

    #0 means take the 48hr
    if target < last_value:
        target = 0
    #1 means take the closing 
    else:
        target = 1

    interpolated_seq = fixed_length(sequence, 20, 'linear')
    interpolated_open = fixed_length(time_since_open_seq, 20, 'linear')
    interpolated_close = fixed_length(time_till_close_seq, 20, 'linear')

    interpolated_all = np.stack((interpolated_seq, interpolated_open, interpolated_close), axis=1)
    X = np.vstack((X, interpolated_all.reshape(1, 20, 3)))
    y = np.vstack((y, np.array([[target]])))

C:\Users\maxbr\AppData\Local\Temp\ipykernel_3928\3888342865.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
C:\Users\maxbr\AppData\Local\Temp\ipykernel_3928\3888342865.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
C:\Users\maxbr\AppData\Local\Temp\ipykernel_3928\3888342865.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
C:\Users\maxbr\AppData\Local\Temp\ipykernel_3928\3888342865.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
C:\Users\maxbr\AppData\Local\Temp\ipykernel_3928\3888342865.py:17: FutureWarning: DataFrame.applymap has been deprecated. Us

In [34]:
print(X.shape)
print(y.shape)

(10483, 20, 3)
(10483, 1)


In [35]:
#80-10-10 split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((8386, 20, 3), (8386, 1), (2097, 20, 3), (2097, 1))

In [36]:
with tf.device('/GPU:0'):
    model = Sequential()
    model.add(InputLayer((20, 3)))
    model.add(SimpleRNN(64, activation='tanh', return_sequences=True))
    model.add(SimpleRNN(32, activation='tanh'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 20, 64)            4352      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                3104      
                                                                 
 dense (Dense)               (None, 16)                528       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 8,129
Trainable params: 8,129
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=25)

Epoch 1/25
263/263 [==============================] - 18s 62ms/step - loss: 1.1462 - accuracy: 0.4950 - val_loss: 0.6967 - val_accuracy: 0.5260
Epoch 2/25
263/263 [==============================] - 15s 57ms/step - loss: 0.6993 - accuracy: 0.5118 - val_loss: 0.6982 - val_accuracy: 0.4907
Epoch 3/25
263/263 [==============================] - 16s 63ms/step - loss: 0.6980 - accuracy: 0.5125 - val_loss: 0.6994 - val_accuracy: 0.5317
Epoch 4/25
263/263 [==============================] - 18s 67ms/step - loss: 0.6968 - accuracy: 0.5114 - val_loss: 0.6932 - val_accuracy: 0.5279
Epoch 5/25
263/263 [==============================] - 17s 66ms/step - loss: 0.6950 - accuracy: 0.5117 - val_loss: 0.6948 - val_accuracy: 0.5289
Epoch 6/25
263/263 [==============================] - 15s 58ms/step - loss: 0.6952 - accuracy: 0.5117 - val_loss: 0.6945 - val_accuracy: 0.5255
Epoch 7/25
263/263 [==============================] - 18s 68ms/step - loss: 0.6940 - accuracy: 0.5174 - val_loss: 0.6937 - val_accuracy: